In [62]:
import pandas as pd
import numpy as np

In [63]:
df_bikes = pd.read_csv('bike_rentals.csv')
df_bikes.head()
# cnt = 각 도시별 일별 자전거 대여 횟수

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,cnt
0,1,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,985
1,2,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,801
2,3,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,1349
3,4,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,1562
4,5,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,1600


In [64]:
# 종속/ 독립 변수 구분
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

In [65]:
# train set 구분
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, test_size=0.2)

### 직접 코딩해서 만드는 경우

In [66]:
from sklearn.tree import DecisionTreeRegressor
# 3개의 Tree 모형을 사용해볼것이기에 대문자 T의 값이 3이라고 생각하고 진행함.

# Initialize Decision Tree Regressor
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)

# Fit tree to training data
tree_1.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [67]:
# 첫번째 단계 종속변수 예측
# Make predictions on training set
y_train_pred = tree_1.predict(X_train)

In [68]:
# Compute residuals , 잔차 구하기
r2_train = y_train - y_train_pred 

# Initialize Decision Tree Regressor
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)

# 구해진 잔차를 종속변수로 받음
# Fit tree to training data
tree_2.fit(X_train, r2_train)

DecisionTreeRegressor(max_depth=2, random_state=2)

In [85]:
# Make predictions on training set
r2_train_pred = tree_2.predict(X_train) # 잔차 예측치

# 잔차 빼기
# Compute residuals
r3_train = r2_train - r2_train_pred

# Initialize Decision Tree Regressor
tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)

# Fit tree to training data
tree_3.fit(X_train, r3_train)

r3_train_pred = tree_3.predict(X_train) # 잔차 예측치

In [86]:
y1_pred = tree_1.predict(X_test) # 종속변수에 대한 예측치(y hat)

y2_pred = tree_2.predict(X_test) # 나머지의 예측치(r1)

y3_pred = tree_3.predict(X_test) # 나머지의 예측치(r2)

y_pred = y1_pred + y2_pred + y3_pred # 최종적인 종속변수 예측치

In [87]:
# Import mean_squared_error 
from sklearn.metrics import mean_squared_error as MSE

# Compute root mean squared error (rmse)
MSE(y_test, y_pred)**0.5

949.085880076671

### Gradient Boosting 모듈 사용하기



In [88]:
from sklearn.ensemble import GradientBoostingRegressor

In [90]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, random_state=2, learning_rate=1.0)
# n_estimators = 3 : 3개의 트리를 쓸 것이다. 

gbr.fit(X_train, y_train)

# Predict test data
y_pred = gbr.predict(X_test)

# Compute root mean squared error (rmse)
MSE(y_test, y_pred)**0.5

949.0858800766707

### T 와 Learning rate의 트레이드 오프 관계 확인
tree를 늘리면 과적합이 될 수 있어 learning rate값을 줄여봐야 함.

In [101]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=.5)
# n_estimators = 3 : 3개의 트리를 쓸 것이다. 

gbr.fit(X_train, y_train)

# Predict test data
y_pred = gbr.predict(X_test)

# Compute root mean squared error (rmse)
MSE(y_test, y_pred)**0.5

772.4851508606969

In [102]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=30, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

886.068525006225

In [103]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

940.9026325887531

In [104]:
# learning_rate = 0.1 (default value)
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

607.2136762123563

### learning rate(shrinkage)가 달라짐에 따라 발생하는 값의 차이 확인(for문을 활용한 Grid search)

In [106]:
learning_rate_values = [0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.5, 1.0]
for value in learning_rate_values:
    gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=value)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Learning Rate:', value, ', Score:', rmse)

Learning Rate: 0.001 , Score: 1635.5979815549326
Learning Rate: 0.01 , Score: 787.1653426300637
Learning Rate: 0.05 , Score: 612.6439751603775
Learning Rate: 0.1 , Score: 607.2136762123563
Learning Rate: 0.15 , Score: 633.1424091448465
Learning Rate: 0.2 , Score: 606.2603007147292
Learning Rate: 0.3 , Score: 666.1495034887553
Learning Rate: 0.5 , Score: 772.4851508606969
Learning Rate: 1.0 , Score: 940.9026325887531


In [ ]:
# l = 0.001의 경우 T를 증가시키면 값을 개선시킬 수 있는데, 단점으로는 연산이 오래걸리고 복잡도가 증가할 수 있음

### Subsample(hyper parameter)

In [107]:
subsamples = [1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]
for subsample in subsamples:
    gbr = GradientBoostingRegressor(max_depth=3, n_estimators=300, subsample=subsample, random_state=2)
    gbr.fit(X_train, y_train)
    y_pred = gbr.predict(X_test)
    rmse = MSE(y_test, y_pred)**0.5
    print('Subsample:', subsample, ', Score:', rmse)

Subsample: 1 , Score: 640.8527800312223
Subsample: 0.9 , Score: 620.1016867594453
Subsample: 0.8 , Score: 619.5686207456519
Subsample: 0.7 , Score: 629.7184792478918
Subsample: 0.6 , Score: 633.2848972033673
Subsample: 0.5 , Score: 627.50379633311
Subsample: 0.4 , Score: 618.554569340628
Subsample: 0.3 , Score: 637.0480801218534
Subsample: 0.2 , Score: 606.9534612335527
Subsample: 0.1 , Score: 638.1277833975638
